In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('../Customized_Components')
from single_pad_transmon_pocket import TransmonPocket_Single
from LaunchpadWirebondCustom import LaunchpadWirebondCustom
from CoupledLineTee import CoupledLineTee


import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond


import pyEPR as epr

import Default_res_params as dp

from qiskit_metal.toolbox_metal.parsing import *
from collections import OrderedDict

In [4]:
import pandas as pd
import numpy as np

### Function that generates anchors for CPW. 

Currently only used to calculate the location of the short to ground component. The RouteMixed or RouteAnchors function does not work

In [5]:
def s_anchors(TQ_options, cpw_options,flip_y = False):
    if flip_y:
        factor = -1
    else:
        factor = 1
    x0 = parse_value('pos_x',TQ_options)
    y0 = parse_value('pos_y',TQ_options)
    prime_gap = parse_value('prime_gap',TQ_options)
    prime_pin = parse_value('prime_width',TQ_options)
    coupling_space = parse_value('coupling_space',TQ_options)
    down_len = parse_value('down_length',TQ_options)

    fillet = parse_value('fillet',cpw_options)
    total_len = parse_value('total_length',cpw_options)

    l = (total_len  - fillet*np.pi)/3

    angle = parse_value('orientation',TQ_options)
    angle = np.deg2rad(angle)

    # print(x0,prime_pin,prime_gap,coupling_space,down_len,l,fillet)
    # d = (prime_pin/2+prime_gap+coupling_space)#-down_len)

    x0 = x0 + (prime_pin/2+prime_gap+coupling_space+down_len)*np.sin(angle)
    y0 = y0 - (prime_pin/2+prime_gap+coupling_space+down_len)*np.cos(angle)*factor

    x1 = x0 + l*np.sin(angle)
    y1 = y0 - l*np.cos(angle)*factor

    x2 = x1 + fillet*np.cos(angle)
    y2 = y1 + fillet*np.sin(angle)*factor

    x3 = x2 - l*np.sin(angle)
    y3 = y2 + l*np.cos(angle)*factor

    x4 = x3 + fillet*np.cos(angle)
    y4 = y3 + fillet*np.sin(angle)*factor

    x5 = x4 + l*np.sin(angle)
    y5 = y4 + l*np.cos(angle)*factor

    result = OrderedDict()
    result[0] = [x1,y1]
    result[1] = [x2,y2]
    result[2] = [x3,y3]
    result[3] = [x4,y4]
    result[4] = [x5,y5]
    return result


# Initialize things by directly importing

In [6]:
gui = MetalGUI(dp.design)
design = dp.design
design.overwrite_enabled = False #This makes it possible to edit the designs later on. 

In [7]:
design.overwrite_enabled = True

In [8]:
design.delete_all_components()

In [9]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

## Specify resonator specs

Hand measured using kLayout

In [10]:
coupling_d = [100,105,140,200,2,15,40,75]
curve_d = [380,380,380,400,300,310,330,340]
ground_plane_d = 10
fillets = [200,200,280,320,200,200,200,240]

prime_pin = 14.18
prime_gap = 8

res_orientation = [(-1)**i*90 for i in range(8)]
res_flip_y = [i%2==0 for i in range(8)]

res_order = [4,2,6,0, 7,1,5,3]


# Put down wirebond pads

In [11]:
wb_x0 = 4.7/2
wb_y0 = 3.5-1.86

In [12]:
Wirebond_opt = Dict(trace_width=design.variables['trace_width'],
                           trace_gap=design.variables['trace_gap'],
                           lead_length='440.5um',
                           pad_width='200um',
                           pad_height='500um',
                           pad_gap='100um',
                           taper_height='100um',
                           layer = str(qubit_layer),
                           pin_space = '200um')

In [13]:
left_lauch_pads = LaunchpadWirebondCustom(design,'wb_left', options = Dict(pos_x = '{}mm'.format(-wb_x0),pos_y = '{}mm'.format(-wb_y0),orientation = '90',**Wirebond_opt))
right_lauch_pads = LaunchpadWirebondCustom(design,'wb_right', options = Dict(pos_x = '{}mm'.format(wb_x0),pos_y = '{}mm'.format(-wb_y0),orientation = '90',**Wirebond_opt))

gui.rebuild()
gui.autoscale()


# Put Down Resonator with Coupled Line Tee

Import resonator parameters. Calculated using coupler_optimization.ipynb with website to optimize frequency and impedance. All the other parameters are measured from dxf file

In [14]:
resonator_params = pd.read_csv('results.csv')
print(resonator_params)

   Unnamed: 0  Frequency          Q  Length  Gap    Pin  coupling_gap
0           0        5.9  1278612.0  5.0535   10  17.73           118
1           1        6.2  1210136.0  4.8072   12  21.27           123
2           2        6.5  1149617.0  4.5836   14  24.82           158
3           3        6.7  1113073.0  4.4457   16  28.37           158
4           4        7.0  1063157.0  4.2536    2   3.55            20
5           5        7.2  1032764.0  4.1344    4   7.09            33
6           6        7.5   990927.0  3.9674    6  10.64            58
7           7        7.9   941290.0  3.7646    8  14.18            93


In [15]:
freqs = resonator_params['Frequency'].values
Qs = resonator_params['Q'].values
gaps = resonator_params['Gap'].values
pins = resonator_params['Pin'].values
coupling_gap = resonator_params['coupling_gap'].values
lengths = resonator_params['Length'].values

Define locations of resonators by defining location of the coupled line tee

In [16]:
res_x = 1.15
delta_x = 3.5-res_x

res_x_all  = np.array([-delta_x*(-1)**i for i in range(8)])

res_y_all = np.array([(5254+5210)/2,(4301+4248)/2,(3448+3387)/2,
             (2395+2325)/2,(2365+2356)/2,(3326+3309)/2,
             (4338+4311)/2,(5249+5214)/2])/1000-3.5

pre define some options

In [17]:
TQ_options = dp.TQ_options
TQ_options['prime_length'] = '120um'
TQ_options['prime_gap'] = design.variables['trace_gap']
TQ_options['prime_width'] = design.variables['trace_width']

## Actual code that puts down CPW and CoupledLineTee

NOTE: need to fix code and use RouteMixed with anchors to route CPWs. Currently using RouteMeander and lack control. 

In [24]:
TQs = []
short_to_grounds = []
cpws = []

connection_cpws = []

for i in range(8):
    curve_d_i = curve_d[i]
    coupling_d_i = coupling_d[i]
    pins_i = pins[i]
    gap_i = gaps[i]
    fillet = fillets[i]

    down_len =  - prime_pin/2 - prime_gap + curve_d_i+fillet*np.pi/2

    TQ_options['down_length'] =  str(down_len) + 'um'
    TQ_options['coupling_space'] = str(ground_plane_d) + 'um'
    TQ_options['second_width'] = str(pins[i]) + 'um'
    TQ_options['second_gap'] = str(gaps[i]) + 'um'
    # TQ_options['coupling_space'] = str(coupling_gap[i])+ 'um'
    TQ_options['pin_drop'] = str(coupling_d[i])+'um'
    TQ_options['orientation'] = str(res_orientation[i])


    TQs.append(CoupledLineTee(design, 'TQ_'+str(i), options=dict(pos_x='{} mm'.format(res_x_all[i]),
                                                pos_y='{} mm'.format(res_y_all[i]), **TQ_options)))

    CPW_options = dp.CPW_options
    CPW_options['pin_width'] = str(pins[i]) + 'um'
    CPW_options['gap_width'] = str(gaps[i]) + 'um'
    CPW_options['total_length'] = str(np.around(lengths[i]-down_len/1000,4)) + 'mm'
    CPW_options['pin_inputs'] = in_inputs=Dict(
                start_pin=Dict(component='TQ_'+str(i), pin='second_end'),
                end_pin=Dict(component='short'+str(i), pin='short'),)

    CPW_options['trace_gap'] = str(gaps[i]) + 'um'
    CPW_options['trace_width'] = str(pins[i]) + 'um'
    CPW_options['lead'] = dict(start_straight = '600 um', end_straight = '400 um')
    CPW_options['fillet'] = '{} um'.format(fillet)
    CPW_options['prevent_short_edges'] = False
    CPW_options['snap'] = True

    anchors = s_anchors(TQs[-1].options, CPW_options,flip_y=res_flip_y[i])
    orientation = parse_value('orientation', TQ_options) + 270
    short_to_grounds.append(ShortToGround(design, 'short'+str(i), options=dict(pos_x='{} mm'.format(anchors[4][0]), pos_y='{} mm'.format(anchors[4][1]), 
                                                        orientation='{}'.format(orientation))))
 
    CPW_options['fillet'] = '{} um'.format(fillet/2.1)
    CPW_options['anchor'] = anchors
    CPW_options['meander'] = dict(spacing= '{}um'.format(fillet), asymmetry= '0um')

    cpws.append(RouteMeander(design, 'cpw'+str(i), options = CPW_options))
                
    

    gui.rebuild()
    gui.autoscale()

### Connect coupled line tees

In [25]:
for i, cpw in enumerate(res_order):
        if i ==7:
                break
        elif i == 3:
                continue
        else:
                trans_ops = dp.trans_options
                trans_ops['pin_inputs'] = {'start_pin': {'component': TQs[cpw].name, 
                                                        'pin': 'prime_end'},
                                        'end_pin': {'component': TQs[res_order[i+1]].name, 
                                                        'pin': 'prime_start'}}
                connection_cpws.append(RouteStraight(design, 
                                                'connection_cpw'+str(cpw), 
                                                options = Dict(**dp.trans_options)))
                gui.rebuild()

In [20]:
anchors = OrderedDict()
anchors[0] = (3.5-1.230,-0.2)
anchors[1] = (3.5-1.230,0.2)

In [21]:
i = 3
trans_ops1 = dp.trans_options
trans_ops1['lead'] = dict(start_straight = '300 um', end_straight = '300 um')
trans_ops1['pin_inputs'] = {'start_pin': {'component': TQs[res_order[i]].name, 'pin': 'prime_end'},
                           'end_pin': {'component': TQs[res_order[i+1]].name, 'pin': 'prime_start'}}
trans_ops1['fillet'] = '90um'
total_len = 0.3*2+4.7 + 0.9*(np.pi-2)
trans_ops1['total_length'] = '{}mm'.format(total_len)
cpw_top = RouteMixed(design, 'connection_cpw'+str(i),options = Dict(anchor = anchors,**trans_ops1))

gui.rebuild()
                

In [26]:
import pyEPR as epr


In [72]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_res = EPRanalysis(design, "hfss")

In [88]:
eig_all,renderer, hfss = dp.init_hfss_sim(nmode = 1, Ljs = [],conv_threshold = 0.005)

In [91]:
data_all = []
for i in range(8):
    eig_all.sim.run(name = 'freq'+str(i), components = ['TQ_'+str(i),'short'+str(i), 'cpw'+str(i)])
    print(eig_all.sim.get_data()['convergence_f'])
    data_all += [eig_all.sim.get_data()]
    renderer.clean_active_design()

INFO 03:01PM [connect_design]: 	Opened active design
	Design:    freq0_hfss [Solution type: Eigenmode]
INFO 03:01PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:01PM [analyze]: Analyzing setup Setup
03:11PM 41s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\Resonator_design\hfss_eig_f_convergence.csv


         re(Mode(1)) [g]
Pass []                 
1               8.432589
2               5.611368
3               4.849948
4               4.986350
5               5.449133
6               5.892581
7               5.731745
8               5.869777
9               5.810271
10              6.004799
11              6.074196
12              6.129394
13              6.147061
14              6.156123
15              6.167358
16              6.161821
17              6.151567
18              6.143547
19              6.121337
20              6.095800
21              6.081045
22              6.082857
23              6.083630
24              6.083065
25              6.084863
26              6.085149
27                   NaN
28                   NaN
29                   NaN
30                   NaN


INFO 03:11PM [connect_design]: 	Opened active design
	Design:    freq1_hfss [Solution type: Eigenmode]
WARNING 03:11PM [connect_setup]: 	No design setup detected.
WARNING 03:11PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:11PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:11PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:11PM [analyze]: Analyzing setup Setup
03:15PM 45s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\Resonator_design\hfss_eig_f_convergence.csv


         re(Mode(1)) [g]
Pass []                 
1               7.610938
2               5.981544
3               4.780081
4               5.080169
5               5.982966
6               5.844419
7               5.776730
8               5.844114
9               5.902867
10              6.026533
11              6.164465
12              6.281965
13              6.334151
14              6.376342
15              6.402313
16              6.410858
17              6.407573
18              6.405744
19              6.399742
20              6.399210
21              6.405114
22              6.411453
23              6.415748
24              6.415922
25                   NaN
26                   NaN
27                   NaN
28                   NaN
29                   NaN
30                   NaN


INFO 03:15PM [connect_design]: 	Opened active design
	Design:    freq2_hfss [Solution type: Eigenmode]
WARNING 03:15PM [connect_setup]: 	No design setup detected.
WARNING 03:15PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:15PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:15PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:15PM [analyze]: Analyzing setup Setup
03:27PM 39s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\Resonator_design\hfss_eig_f_convergence.csv


...

         re(Mode(1)) [g]
Pass []                 
1               7.163643
2               7.376437
3               7.053345
4               6.217049
5               6.132299
6               5.968225
7               6.073555
8               6.240299
9               6.282736
10              6.503312
11              6.758894
12              6.876777
13              6.994338
14              7.105269
15              7.161533
16              7.233213
17              7.283588
18              7.302716
19              7.301012
20              7.296875
21              7.289814
22              7.286405
23              7.285712
24              7.290811
25              7.296543
26              7.298831
27              7.300686
28              7.302358
29              7.301882
30              7.301071


INFO 04:38PM [connect_design]: 	Opened active design
	Design:    freq6_hfss [Solution type: Eigenmode]
WARNING 04:38PM [connect_setup]: 	No design setup detected.
WARNING 04:38PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:38PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:38PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:38PM [analyze]: Analyzing setup Setup
04:40PM 02s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\Resonator_design\hfss_eig_f_convergence.csv


         re(Mode(1)) [g]
Pass []                 
1               6.661672
2               7.681683
3               8.380624
4               7.895405
5               7.210829
6               6.855589
7               6.740985
8               6.978280
9               7.152668
10              7.246251
11              7.340316
12              7.505587
13              7.580118
14              7.605981
15              7.633404
16              7.648026
17              7.649765
18              7.645706
19              7.639381
20              7.639051
21                   NaN
22                   NaN
23                   NaN
24                   NaN
25                   NaN
26                   NaN
27                   NaN
28                   NaN
29                   NaN
30                   NaN


INFO 04:40PM [connect_design]: 	Opened active design
	Design:    freq7_hfss [Solution type: Eigenmode]
WARNING 04:40PM [connect_setup]: 	No design setup detected.
WARNING 04:40PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:40PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:40PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:40PM [analyze]: Analyzing setup Setup
04:44PM 56s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\Resonator_design\hfss_eig_f_convergence.csv


         re(Mode(1)) [g]
Pass []                 
1              10.964645
2              11.420364
3               9.559498
4               8.715186
5               8.837511
6               8.872934
7               8.341192
8               8.566022
9               8.299867
10              7.897126
11              8.103432
12              8.056671
13              8.109619
14              8.112742
15              8.123724
16              8.114692
17              8.115318
18              8.111940
19              8.124572
20              8.125425
21              8.130102
22              8.137707
23              8.148938
24              8.151120
25              8.151152
26                   NaN
27                   NaN
28                   NaN
29                   NaN
30                   NaN


In [93]:
data_all

[{'sim_setup_name': 'Setup',
  'convergence_t':              Solved Elements  Max Delta Freq. %
  Pass Number                                    
  1                        545                NaN
  2                        718          33.456000
  3                        936          13.569000
  4                       1217           2.812400
  5                       1589           9.281000
  6                       2067           8.137900
  7                       2657           2.729500
  8                       3432           2.408200
  9                       4473           1.013800
  10                      5714           3.348000
  11                      7435           1.155700
  12                      9672           0.908720
  13                     12578           0.288240
  14                     16357           0.147420
  15                     21266           0.182500
  16                     27650           0.089783
  17                     35947           0.166400
  18

In [92]:
eig_all.sim.get_data()

{'sim_setup_name': 'Setup',
 'convergence_t':              Solved Elements  Max Delta Freq. %
 Pass Number                                    
 1                        541                NaN
 2                        595           4.156300
 3                        674          16.294000
 4                        886           8.832200
 5                       1154           1.403600
 6                       1502           0.400830
 7                       1966           5.992900
 8                       2567           2.695400
 9                       3339           3.107100
 10                      4346           4.852400
 11                      5657           2.612400
 12                      7364           0.577040
 13                      9577           0.657190
 14                     12455           0.038504
 15                     16193           0.135370
 16                     21055           0.111190
 17                     27376           0.007720
 18                     

In [73]:
hfss = eig_res.sim.renderer

In [78]:
hfss.start()

INFO 02:17PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:17PM [load_ansys_project]: 	Opened Ansys App
INFO 02:17PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 02:17PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/slab/Documents/Ansoft/
	Project:   Project46
INFO 02:17PM [connect_design]: No active design found (or error getting active design).
INFO 02:17PM [connect]: 	 Connected to project "Project46". No design detected


True

In [80]:
for i in range(8):
    hfss.activate_ansys_design(f"Readout {i}", 'eigenmode')
    hfss.render_design(['TQ_'+str(i),'short'+str(i), 'cpw'+str(i)], [])


02:18PM 24s WARNING [activate_ansys_design]: The design_name=Readout 0 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 02:18PM [connect_design]: 	Opened active design
	Design:    Readout 0 [Solution type: Eigenmode]
WARNING 02:18PM [connect_setup]: 	No design setup detected.
WARNING 02:18PM [connect_setup]: 	Creating eigenmode default setup.
INFO 02:18PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 02:18PM [__del__]: Disconnected from Ansys HFSS
02:18PM 27s WARNING [activate_ansys_design]: The design_name=Readout 1 was not in active project.  Designs in active project are: 
['Readout 0'].  A new design will be added to the project.  
INFO 02:18PM [connect_design]: 	Opened active design
	Design:    Readout 1 [Solution type: Eigenmode]
WARNING 02:18PM [connect_setup]: 	No design setup detected.
WARNING 02:18PM [connect_setup]: 	Creating eigenmode default setup.
INFO 02:18PM [get_setup]: 	Op

In [83]:
hfss.options

{'Lj': '13nH',
 'Cj': 0,
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'max_mesh_length_port': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'x_buffer_width_mm': 0.5,
 'y_buffer_width_mm': 0.5,
 'wb_threshold': '72um',
 'wb_offset': '0um',
 'wb_size': 1.6,
 'wb_height': 0.7,
 'plot_ansys_fields_options': {'name': 'NAME:Mag_E1',
  'UserSpecifyName': '0',
  'UserSpecifyFolder': '0',
  'QuantityName': 'Mag_E',
  'PlotFolder': 'E Field',
  'StreamlinePlot': 'False',
  'AdjacentSidePlot': 'False',
  'FullModelPlot': 'False',
  'IntrinsicVar': "Phase='0deg'",
  'PlotGeomInfo_0': '1',
  'PlotGeomInfo_1': 'Surface',
  'PlotGeomInfo_2': 'FacesList',
  'PlotGeomInfo_3': '1'}}

In [77]:
hfss.close()

Warning! 6 COM references still alive
Ansys will likely refuse to shut down


True

In [64]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 20
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")


Number of eigenmodes to find             = 1
Number of simulation passes              = 20
Convergence freq max delta percent diff  = 0.1



The ideal way is to connect the Tee with launcher pad using a CPW, but this thing does not export properly so I used a skechy fix. Currently working fine.

In [22]:
# connection_cpw_ops = {'trace_width': '14.18 um',
#  'trace_gap': '8 um',
#  'lead': {'start_straight': '300 um', 'end_straight': '300 um'},
#  'layer': 5,
#  'hfss_wire_bonds': False,
#  'q3d_wirebonds': False,
#  }
# connection_cpw_ops['pin_inputs'] = {'end_pin': {'component':'TQ_4','pin': 'prime_start'},'start_pin': {'component': 'wb_left', 'pin': 'tie'}}
# # connection_cpws.append(RouteStraight(design,'connection_cpw'+str(cpw), options = Dict(**dp.trans_options)))

# cpw_left_launch = RouteStraight(design, 'cpw_left_launch', options = Dict(**connection_cpw_ops))

# connection_cpw_ops['pin_inputs'] = {'end_pin': {'component':'TQ_3','pin': 'prime_end'},'start_pin': {'component': 'wb_right', 'pin': 'tie'}}
# cpw_right_launch = RouteStraight(design, 'cpw_right_launch', options = Dict(**connection_cpw_ops))

# gui.rebuild()

In [23]:
# design.chips.main.size['size_x'] = '10mm'
# design.chips.main.size['size_y'] = '10mm'
# gui.rebuild()
a_gds = design.renderers.gds

In [24]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

In [25]:
a_gds.options['max_points'] = 3000
a_gds.options['fabricate'] = False

In [26]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
a_gds.options['cheese']['view_in_file']['main'][1] = False
a_gds.options['no_cheese']['view_in_file']['main'][1] = False

In [27]:
# a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['cheese']['view_in_file']['main']['1'] = True
# a_gds.options['no_cheese']['view_in_file']['main']['1'] = True



# a_gds.options['fabricate'] = False
a_gds.options.tolerance = '0.0000005'
a_gds.options.precision = '0.0000000005'

In [28]:
a_gds.options['cheese']

{'datatype': '100',
 'shape': '0',
 'cheese_0_x': '25um',
 'cheese_0_y': '25um',
 'cheese_1_radius': '100um',
 'view_in_file': {'main': {1: False,
   5: True,
   20: False,
   31: False,
   30: False}},
 'delta_x': '100um',
 'delta_y': '100um',
 'edge_nocheese': '200um'}

In [29]:
a_gds.options.cheese.edge_nocheese = '150um'
a_gds.options.no_cheese.buffer = '200um'
a_gds.options.cheese.cheese_1_radius = '100um'

In [30]:
# gui.rebuild()
a_gds.export_to_gds('resonator_design.gds')

05:02PM 33s WARNING [_import_junction_gds_file]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"/Users/wendy/Desktop/Wendy-qiskit-code/resources".


ground_main_5


1